# Threads para processar em tempo real

A ideia aqui é rodar o sistema em tempo real antes de passar pra um .py msm.

In [ ]:
import sys
import pyaudio
import numpy as np
import warnings
import threading
from time import time, sleep

sys.path.append("/home/pi/Programming/IC2019/Raspberry/Python/")

from delayAndSum import delayAndSum as DaS
from extracaoFeatures import extrairFeaturesUnicoFrame as ExtrairFeatures
from iniciarNormalizadorEClassificador import main as IniciarObjetos

warnings.filterwarnings("ignore")

### Verificando o dispositivo que vamos usar pra gravar

In [ ]:
def listarDispositivos():
    p = pyaudio.PyAudio()
    info = p.get_host_api_info_by_index(0)
    numdevices = info.get('deviceCount')

    for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

In [ ]:
listarDispositivos()

### Definição de funções

##### Função para realizar todo o processamento em uma janela

A função receberá uma janela gravada, aplicará o **Delay and Sum**, extrairá as **features** do sinal beamformado, **normalizará** o vetor de features obtido, e irá **classificar** o dado gerado.

In [ ]:
# def processarJanela(arraySinais, idJanela, freqAmostragem, objNormalizador, objClassificador, verbose=True):
    
#     # VERBOSE
#     if verbose == True:
#         # INICIANDO A MEDICAO DE TEMPO
#         tempoInicio = time()
#         stringID = "(id:" + str(idJanela) + ")"
#         print(stringID, "Iniciando o processamento")      
    
#     # DELAY AND SUM
#     sinalDaS = DaS(arraySinais)

#     # EXTRACAO DE FEATURES
#     features = ExtrairFeatures(sinalDaS, freqAmostragem)

#     # NORMALIZANDO AS FEATURES
#     features = objNormalizador.transform([features])[0]

#     # CLASSIFICACAO
#     predicao = objClassificador.predict([features])[0]
    
#     # VERBOSE
#     if verbose == True:
#         # FINALIZANDO A MEDICAO DE TEMPO
#         tempoFim = time()        
#         print(stringID, "Classificado como", predicao)
#         print(stringID, "Processamento finalizado em", tempoFim - tempoInicio, "s")

##### Função para gravar uma janela com o dispositivo

A função abaixo deverá gravar uma janela e depois enviá-la para a função anterior. A função anterior deve ser rodada como uma thread.

In [ ]:
# def gravarJanela(idJanela, objNormalizador, objClassificador, idDispositivoGravacao, tempoJanela=0.2, freqAmostragem=16000, profundidadeBytes=2, qtdCanais=4, verbose=True):

#     assert profundidadeBytes == 2, "Profundidade de bytes diferente de 2! Você deve alterar o sistema manualmente."
    
#     # DEFININDO O TAMANHO DA JANELA EM AMOSTRAS
#     tamanhoJanela = int(tempoJanela * freqAmostragem)
    
#     # INSTANCIANDO UM OBJ PY AUDIO E MANDANDO OS PARAMETROS INICIAIS
#     objPyAudio = pyaudio.PyAudio()
#     stream = objPyAudio.open(
#         input_device_index = idDispositivoGravacao,
#         rate               = freqAmostragem,
#         format             = objPyAudio.get_format_from_width(profundidadeBytes),
#         channels           = qtdCanais,
#         input              = True    
#     )
    
#     # VERBOSE 
#     if verbose == True:
#         stringID = "(id:" + str(idJanela) + ")"
#         print(stringID, "Iniciando gravação. Timestamp:", time())

#     # GRAVANDO A JANELA
#     janelaBinaria = stream.read(tamanhoJanela)
    
#     # VERBOSE 
#     if verbose == True:
#         print(stringID, "Gravação finalizada. Timestamp:", time())

#     # MATANDO OS OBJETOS PRA LIMPAR MEMORIA
#     stream.stop_stream()
#     stream.close()
#     objPyAudio.terminate()
        
#     # CONVERTO A JANELA PRA INT 16        
#     janelaInt16 = np.fromstring(janelaBinaria, dtype=np.int16)
    
#     # COMO TEM OS 4 MICS NA JANELA, VOU DAR UM RESHAPE PRA CADA MIC FICAR EM UMA LINHA
#     janelaInt16 = janelaInt16.reshape((tamanhoJanela, qtdCanais)).T
    
#     # GARANINDO QUE A DIMENSIONALIDADE ESTA CORRETA
#     assert janelaInt16.shape[0] == qtdCanais and janelaInt16.shape[1] == tamanhoJanela, "Erro na dimensionalidade da janela gravada!"
    
#     # MANDANDO A JANELA GRAVADA APRA O PROCESSAMENTO
#     processarJanela(janelaInt16, idJanela, freqAmostragem, objNormalizador, objClassificador, verbose)
#     #objThread = threading.Thread(target=processarJanela, args=(janelaInt16, idJanela, freqAmostragem, objNormalizador, objClassificador, verbose))
#     #objThread.start()

In [ ]:
def main(idJanela, idDispositivoGravacao, tempoJanela, freqAmostragem, profundidadeBytes, qtdCanais):
    
    # DEFININDO O TAMANHO DA JANELA EM AMOSTRAS
    tamanhoJanela = int(tempoJanela * freqAmostragem)
    
    # VERBOSE
    stringID = "(id:" + str(idJanela) + ")"

    # INSTANCIANDO UM OBJ PY AUDIO E MANDANDO OS PARAMETROS INICIAIS
    objPyAudio = pyaudio.PyAudio()
    
    # TENTANDO ABRIR O STREAM ATE IR
    stream = None
    while stream == None:
        print(stringID, "Tentando abrir uma stream")
        try:
            stream = objPyAudio.open(
                input_device_index = idDispositivoGravacao,
                rate               = freqAmostragem,
                format             = objPyAudio.get_format_from_width(profundidadeBytes),
                channels           = qtdCanais,
                input              = True
            )
        except:
            pass

    # VERBOSE
    print(stringID, "Iniciando gravação. Timestamp:", time())

    # GRAVANDO A JANELA
    janelaBinaria = stream.read(tamanhoJanela)

    # MATANDO OS OBJETOS PRA LIMPAR MEMORIA
    stream.stop_stream()
    stream.close()
    objPyAudio.terminate()
    del stream
    del objPyAudio
    
    # INICIANDO A MEDICAO DE TEMPO DE PROCESSAMENTO
    tempoInicio = time()
    print(stringID, "Iniciando o processamento")

    # CONVERTO A JANELA PRA INT 16
    janelaInt16 = np.fromstring(janelaBinaria, dtype=np.int16)

    # COMO TEM OS 4 MICS NA JANELA, VOU DAR UM RESHAPE PRA CADA MIC FICAR EM UMA LINHA
    janelaInt16 = janelaInt16.reshape((tamanhoJanela, qtdCanais)).T    

    # DELAY AND SUM
    sinalDaS = DaS(janelaInt16)

    # EXTRACAO DE FEATURES
    features = ExtrairFeatures(sinalDaS, freqAmostragem)

    # NORMALIZANDO AS FEATURES
    features = objNormalizador.transform([features])[0]

    # CLASSIFICACAO
    predicao = objClassificador.predict([features])[0]

    # FINALIZANDO A MEDICAO DE TEMPO
    tempoFim = time()
    print(stringID, "Classificado como", predicao)
    print(stringID, "Processamento finalizado em", tempoFim - tempoInicio, "s")
    if tempoFim - tempoInicio >= tempoJanela:
        print(stringID, "O PROCESSAMENTO LEVOU MAIS TEMPO QUE O TAMANHO DA JANELA!")

### Parâmetros iniciais

In [ ]:
idDispositivoGravacao = 0
tempoJanela           = 0.200
freqAmostragem        = 16000
profundidadeBytes     = 2
qtdCanais             = 4
caminhoCSVDataset     = "/home/pi/Datasets/SESA_v2_16kHz_16bits_200ms_58features_desescalonado_remocaoSilencio.csv"

### Rodando

In [ ]:
objClassificador, objNormalizador = IniciarObjetos(caminhoCSVDataset, classificador=None, verbose=True)

In [ ]:
i = 1
tempoJanelaTempoReal = 0.2 #tempoJanela

while i < 100:
    
    # INICIANDO JANELA PRINCIPAL
    objThread = threading.Thread(target=main, args=(i, idDispositivoGravacao, tempoJanelaTempoReal, freqAmostragem, profundidadeBytes, qtdCanais))
    objThread.start()
    
    # DELAY PARA COMECAR A JANELA DE SOBREPOSICAO
    i += 1
    sleep(tempoJanelaTempoReal)